In [ ]:
import pandas as pd
import numpy as np
import requests

# Web Scrapping

Muitas vezes os dados que queremos não estão disponibilizados através de APIs, apenas sites. Neste momento precisamos recorrer às ferramentas de **Web Scrapping**!

*Web scrapping* é a extração de informação estruturada a partir de paginas na internet: por exemplo, podemos extrair todos os artigos de um jornal que mencionem um certo produto, ou então as informações de uma série de tabelas da Wikipedia.

Hoje vamos aprender como utilizar as bibliotecas BeautifulSoup e Selenium para extrair informações a partir de links específicos, realizar buscas e navegar páginas.

# Conhecendo o BeautifulSoup

Vamos começar extraindo informações básicas a partir de uma notícia do portal UOL. O primeiro passo é utilizar a biblioteca `requests` para *baixar* o html da página:

In [ ]:
url = "https://www.uol.com.br/esporte/futebol/ultimas-noticias/2022/01/22/em-1995-decisao-na-base-entre-palmeiras-x-sp-terminou-em-morte-no-pacaembu.htm"


In [ ]:
response = requests.get(url)
html_str = response.text
type(html_str)


In [ ]:
print(html_str[0:1000])


In [ ]:
html_bytes = response.content
type(html_bytes)


In [ ]:
print(html_bytes[0:1000])


## Transfome em sopa

A página extraída acima é apenas um string: o código HTML. Para utiliza-la dentro do Python, precisamos de um **parser**: um conjunto de funções que nos permite **interpretar** este código HTML e extrair informações relevantes. A biblioteca *BeautifulSoup* implementa um **parser** de HTML dentro do Python, dando acesso à arvore de tags (`<head>`, `<link ...> `, etc).

Para utilizar este **parser** precisamos entender um pouco da estrutura de um arquivo HTML. Mas antes vamos utilizar o BeautifulSoup para deixar o *print* de nosso HTML mais organizado:

In [ ]:
!pip3 install bs4

In [ ]:
from bs4 import BeautifulSoup


In [ ]:
soup = BeautifulSoup(html_bytes)


In [ ]:
type(soup)


In [ ]:
print(soup.prettify())


## Encontrando Tags

Um arquivo HTML é estruturado em **tags**: marcações com o formato `<nome_do_tag>` e `</nome_do_tag>`. A primeira denota o **inicio do conteúdo** do tag, a segunda o **fim do conteúdo**. Vamos entender como um **tag simples** funciona analisando o conteúdo do `<title>`.

Para encontrar todas as ocorrências de um **tag** pelo seu **nome** utilizamos o método `.find_all()`:

In [ ]:
soup.find_all("title")


O método nos retornou todas as ocorrências de `<title>` no HTML (uma só no caso) em uma lista. Vamos ver o que essa lista contém:

In [ ]:
title = soup.find_all("title")[0]
print(type(title))


O objeto `Tag` da BeautifulSoup contém todas as informações de um tag: atributos, links, conteúdo... Por enquanto vamos olhar o conteúdo desse tag (o que está entre `<title>` e `< \title>`) utilizando o atributo `.text`:

In [ ]:
title.text


In [ ]:
type(title.text)


### **Tags** com mais que uma ocorrência 

A maior parte dos **tags** ocorrem múltiplas vezes em um documento. Vamos buscar um **tag** com essa característica.

In [ ]:
headers = soup.find_all("h3")
print(headers)


In [ ]:
print(headers[0])


In [ ]:
print(headers[0].text)


In [ ]:
# EXERCICIO
# Utilize uma list comprehension para criar uma lista com o texto de todos os headers h3


### Buscando múltiplos **tags**

Além de buscar **tags** um a um, podemos utilizar o método `.find_all()` para encontrar todas as ocorrências de uma lista de tags:

In [ ]:
lista_tags = ["h1", "h2", "h3"]
todos_headers = soup.find_all(lista_tags)
print(todos_headers)


Podemos utilizar o atributo `.name` para determinar qual o tipo de cada tag em nossa lista:

In [ ]:
print(todos_headers[0].name)
print(todos_headers[0].text)


In [ ]:
# EXERCICIO
# Utilize uma list comprehension para criar uma lista
# de uplas (tipo do tag, conteúdo)


### Buscando links

Um **tag** específico muito útil na construção de **web crawlers** é o `<a>`. Este **tag** contém os hiper-links de uma página HTML. Vamos utilizar a BeautifulSoup para extrair todas os links de nossa notícia.

In [ ]:
tag_a = soup.find_all("a")


In [ ]:
tag_a[0:5]


In [ ]:
first_link = tag_a[0]


In [ ]:
first_link


O atributo `.text` não extrai o URL do link, apenas o texto que é exibido para o usuário:

In [ ]:
first_link.text


Se olharmos o *string* do **tag** poderemos entender melhor porque isso acontece. O URL em si está **dentro da declaração do tag**:

    <a data-audience-click="" href="https://www.ingresso.com/?utm_source=uol.com.br&amp;utm_medium=barrauol&amp;utm_campaign=linkfixo_barrauol&amp;utm_content=barrauol-link-ingressocom&amp;utm_term=barrauol-ingressocom">

Dentro da declaração do **tag** podemos ver algo parecido com a declaração de variáveis:

    data-audience-click=""

e

    href="https://www.ingresso.com/?utm_source=uol.com.br&amp;utm_medium=barrauol&amp;utm_campaign=linkfixo_barrauol&amp;utm_content=barrauol-link-ingressocom&amp;utm_term=barrauol-ingressocom"

Cada uma dessas *variáveis* é um **atributo do tag** e para extrai-las utilizaremos o atributo `.attrs`:

In [ ]:
first_link.attrs


Podemos ver que os atributos de um **tag** são retornados como um **dict**! Se quisermos acessar uma **variável** específica podemos faze-lo através do nome desta variável:

In [ ]:
first_link.attrs["href"]


In [ ]:
# EXERCICIO
# Utilize um loop para extrair todos os URLs de
# tags <a>


## **Tags** hierárquicos

Até agora todos os **tags** que vimos eram **simples**, ou seja, não continham em seu conteúdo outros **tags**. Muitas vezes queremos extrair informações *navegando* a página **bloco a bloco**.

Vamos aprender a utilizar o **inspetor de código** de um web browser para navegar blocos de tags para extrair informações complexas. Neste exemplo vamos reconstruir um tabela da Wikipedia com um DataFrame.

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy"


In [ ]:
response = requests.get(url)
html = response.content


In [ ]:
soup = BeautifulSoup(html)


In [ ]:
print(soup.prettify())


Vamos acessar o link [List of European countries by life_expectancy](https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy) para entender como podemos utilizar o inspetor de código fonte para descobrir onde em nosso HTML está nossa tabela!

### Extraindo a tabela completa

Com o inspetor podemos ver que as tabelas desta página estão todas dentro de tags `table`. 

In [ ]:
table_wiki_raw = soup.find_all("table")


In [ ]:
len(table_wiki_raw)


Infelizmente, esta **tag** é muitas vezes utilizada como elemento de formatação de páginas (especialmente em página antigas).

Podemos utilizar o **argumento** `attrs =` do método `.find_all()` para **filtrar** os **tags** extraídos a partir dos **valores de seus atributos**.

O atributo `class` é um ótimo candidato em varias ocasiões para esse tipo de filtro. No caso da nossa tabela atual podemos ver que a classe é:

    wikitable sortable static-row-numbers plainrowheaders srn-white-background jquery-tablesorter

Vamos ver como utilizar o `attrs =` para realizar esse filtro.

In [ ]:
table_wiki_raw = soup.find_all("table", attrs={"class": "wikitable"})


In [ ]:
len(table_wiki_raw)


In [ ]:
print(table_wiki_raw[0])


### Navegando um **tag hierárquico**

Além de poder percorrer o HTML completo utilizando o método `.find_all()`, podemos fazer buscas dentro de cada **tag**! Isso nos permite **localizar** a busca em um bloco específico delimitado por **tags**.

No nosso caso atual, podemos ver que cada tabela extraída é composta por **três tags**: `<thead>`, `<tbody>` e `<tfoot>`. Como queremos extrair o **corpo** da tabela, vamos investigar o tag `tbody`

In [ ]:
table_wiki = table_wiki_raw[3].find("tbody")


In [ ]:
print(table_wiki.prettify())


Podemos ver tanto no inspetor quanto no print acima que o corpo da tabela é composto por múltiplos tags `<tr>`: se olharmos com cuidado veremos que cada `<tr>` é uma linha de nossa tabela! Vamos extrair uma linha em particular para ver do que ela é composta.

In [ ]:
table_rows = table_wiki.find_all("tr")


In [ ]:
print(table_rows[1].prettify())


Cada linha de nossa tabela é composta por uma série de tags `<td>` - e cada um destes contém as informações de uma célula. Finalmente chegamos no elemento que contém os dados da tabela!

In [ ]:
first_row = table_rows[1].find_all("td")


In [ ]:
first_row[2].text


In [ ]:
[elemento.text.strip() for elemento in first_row]


Vamos juntar todas as etapas acima em um loop (ou list comprehension) para construir nosso DataFrame!

In [ ]:
# EXERCICIO
# Construir um DataFrame com os dados da tabela de Expectativa de Vida na Europa da Wikipedia
# BONUS - faça isso com um list comprehension!
# BONUS - contrua uma função que receba um link da Wikipedia com uma tabela e retorne um DataFrame (teste em outra tabela)


# Criando um WebCrawler

Um **webcrawler** é uma aplicação que extrai informações estruturadas a partir de multiplos sites de forma autonôma. Vamos construir um **webcrawler** para extrair um artigo (artigo origem) do jornal Guardian e qualquer outro artigo referenciado no texto do artigo origem (artigos filhos).

## Extraindo o Artigo Origem

O primeiro passo do nosso *crawler* é encontrar o corpo principal do artigo - caso contrário *puxaremos* links de propagandas, artigos relacionados, cabeçalho, etc...

In [ ]:
url = "https://www.theguardian.com/world/2022/aug/18/fires-and-explosions-reported-at-military-targets-in-russia-and-crimea"
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html)


Vamos começar buscando algum **tag** que represente o corpo do artigo:

In [ ]:
artigo = soup.find("article")
print(artigo.text)


Agora que encontramos um **tag** com o corpo do artigo precisamos encontrar links dentro deste corpo:

In [ ]:
links_artigo = artigo.find_all("a", attrs={"data-link-name": "in body link"})
len(links_artigo)


Podemos utilizar list comprehensions para visualizar para onde esses links nos levarão:

In [ ]:
[link["href"] for link in links_artigo]


Sempre que precisamos construir um loop para executar um bloco de código sobre os elementos de uma lista devemos **testar** esse bloco de código em um elemento particular da lista antes de contruir o loop completo:

In [ ]:
url_filho = links_artigo[0]["href"]


Da mesma forma que extraímos o artigo original, podemos extrair o artigo filho a partir do link extraído:

In [ ]:
response_filho = requests.get(url_filho)
html_filho = response_filho.content
soup_filho = BeautifulSoup(html_filho)


In [ ]:
artigo_filho = soup_filho.find("article").text
print(artigo_filho)


Agora podemos consolidar nosso código para executar o bloco acima sobre todos os *links* extraídos do artigo origem (adicionando algumas camadas de segurança para que nosso *crawler* navegue tranquilamente):

In [ ]:
lista_artigos = []
lista_links = [link["href"] for link in links_artigo]
for link in lista_links:
    if link.find("guardian") >= 0:
        try:
            response_filho = requests.get(link)
            html_filho = response_filho.content
            soup_filho = BeautifulSoup(html_filho)
            lista_artigos.append(soup_filho.find("article").text)
        except AttributeError:
            continue


In [ ]:
lista_artigos[2]


In [ ]:
# EXERCICIO
# Transforme o código acima em uma função que recebe um link de artigo do Guardian como argumento
# e retorna uma lista com o texto do artigo original e o texto de quaisquer artigos do Guardian
# linkados no texto do artigo original
# BONUS - crie uma função que faça isso em profundidade maior que um: além do artigo original e dos artigos
# filhos (artigos linkados no artigo original) trazer os artigos netos (artigos linkados nos artigos filhos)
# BONUS BONUS - contrua uma função capaz de extrair uma profundidade arbitraria de artigos (filhos, netos, bisnetos...)
# a partir de um parametro 'depth' (0 = original, 1 = filhos, 2 = netos, 3 = bisnetos, etc)


# Conhecendo o Selenium

Embora o BeautifulSoup seja uma biblioteca ótima e simples para extrair dados de páginas (o complicado são os html's...), nem sempre conseguimos usa-la: muitas páginas utilizam, hoje em dia, tecnologias incompatíveis com a arquitetura do BeautifulSoup.

Páginas que são dinâmicas (especificamente, páginas que usem tecnologias client-side, como React.js) não podem ser mineradas diretamente.

Para tratar destas páginas precisamos utilizar outra biblioteca: Selenium. Enquanto na BeautifulSoup carregamos o HTML da página original para dentro do Python, com a Selenium **simularemos o ato de navegação**.

In [ ]:
!pip3 install selenium
!pip3 install webdriver-manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains


Com a Selenium, precisamos utilizar um WebDriver - um navegador específico através do qual extraíremos as informações desejadas. Para nossa aula de hoje vamos utilizar o Chrome:

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


Vamos utilizar nosso `driver` para navegar até uma página teste:

In [ ]:
driver.get("https://testpages.herokuapp.com/styled/index.html")


## Navegando utilizando XPATHs

O forma mais fácil de navegar uma página utilizando Selenium é através de XPATHs: identificadores únicos dos elementos de uma página. Vamos ver como podemos descobrir um XPATH de um elemento particular e como utilizar o nosso `driver` para interagir com o elemento.

In [ ]:
link_htmlformtest = driver.find_element(
    By.XPATH, "/html/body/div/ul[1]/li[7]/ul/li[1]/a"
)


O método `.find_element()` nos permite encontrar elementos em uma página (parecido com o `.find_all()` do BS) - mas por si só não nos diz muito sobre o elemento:

In [ ]:
link_htmlformtest


Podemos buscar os atributos do elemento utilizando o método `.get_attribute()` - vamos utilizar este método para extrair o URL do link associado ao elemento:

In [ ]:
link_htmlformtest.get_attribute("href")


Até agora tudo está muito parecido com a BS - agora vamos interagir com a página, *clicando* virtualmente no link:

In [ ]:
link_htmlformtest.click()


### Interagindo com Formulários

Um dos principais usos para o Selenium é conseguir preencher de forma programática formulários em páginas. Vamos continuar utilizando nosso driver para preencher o formulário na página para qual navegamos.

In [ ]:
input_username = driver.find_element(
    By.XPATH, "/html/body/div/div[3]/form/table/tbody/tr[1]/td/input"
)


Podemos utilizar o método `.send_keys()` para *preencher* um formulário: 

In [ ]:
input_username.send_keys("aaaa")


e o método `.clear()` para *limpar* o formulário:

In [ ]:
input_username.clear()


Como a variável `input_username` ainda é o mesmo elemento (o campo **Username**), podemos voltar a preencher o formulário:

In [ ]:
nome_usuario = "pedrotechel"
input_username.send_keys(nome_usuario)


Elementos clicáveis, como botões ou checkboxes podem ser acessados através do método click:

In [ ]:
input_opt1 = driver.find_element(
    By.XPATH, "/html/body/div/div[3]/form/table/tbody/tr[5]/td/input[1]"
)
input_opt1.click()


In [ ]:
input_rdbx1 = driver.find_element(
    By.XPATH, "/html/body/div/div[3]/form/table/tbody/tr[6]/td/input[1]"
)
input_rdbx1.click()


## Exemplo Prático - Mercado Livre

Vamos construir um *webcrawler*, baseado em Selenium, para buscar preços de azeite no Mercado Livre. Plataformas de e-commerce quase sempre utilizam tecnologias que impossibilitam a utilização do BeautifulSoup!

Além de *apontar* o driver para a página principal do Mercado Livre, vamos utilizar o método `.implicitly_wait()` para que o driver *espere* um tempo até todos os elementos da página renderizarem:

In [ ]:
driver.get("https://www.mercadolivre.com.br/")
driver.implicitly_wait(10)


Agora vamos interagir com a barra de busca para procurar azeites. Muitas barras de busca tem testos pré-preenchidos, então antes de continuarmos com nossa busca vamos limpar a barra utilizando o método `.clear()`:

In [ ]:
barra_busca = driver.find_element(By.XPATH, "/html/body/header/div/div[2]/form/input")
barra_busca.clear()


Com a barra limpa, podemos utilizar o método `.send_keys()` para preenche-la com o nome do produto que queremos buscar (`"azeite"`) e confirmar a busca utilizando o objeto `Keys.ENTER` (simulando digitar "azeite" e apertar *enter* na barra de busca): 

In [ ]:
barra_busca.send_keys("azeite")
barra_busca.send_keys(Keys.ENTER)


Até agora tranquilo! Para continuarmos, no entanto, precisaremos extrair TODOS os preços de produtos. Para isso utilizaremos o método `.find_elements()` (no plural) e, ao invés de utilizar o XPATH, buscaremos pelo **tag** e sua **classe**. Vamos começar encontrando todas as *caixas de produto*

In [ ]:
lista_elem_produto = driver.find_elements(
    By.CLASS_NAME, "ui-search-result__content-wrapper"
)
len(lista_elem_produto)


A lista construída acima contém `WebElements` - da mesma forma que podemos fazer buscas dentro de **tags** hierarquicas utilizando o BS, podemos fazer buscas dentro de diferentes `WebElements` utilizando o Selenium:

In [ ]:
lista_elem_preco = []
for produto in lista_elem_produto:
    caixa_preco = produto.find_element(By.CLASS_NAME, "price-tag-amount")
    lista_elem_preco.append(caixa_preco)


Vamos utilizar o atributo `.text` para extrair o conteúdo de cada uma das caixas de preço (primeiro testando com um elemento da lista):

In [ ]:
preco_teste = lista_elem_preco[0]
preco_teste.text


O preço extraído acima parece não bater com o preço da página! No entanto, se prestarmos atenção, veremos que este é o preço sem desconto do produto (preço cheio).

Vamos alterar nosso loop para extrair todos os preços de cada produto (cheio e descontado):

In [ ]:
lista_elem_preco = []
for produto in lista_elem_produto:
    caixa_preco = produto.find_elements(By.CLASS_NAME, "price-tag-amount")
    lista_elem_preco.append(caixa_preco)


In [ ]:
lista_elem_preco[0]


Temos 3 preços associados à este produto... Vamos analisar o que cada um deles representa:

In [ ]:
[preco.text for preco in lista_elem_preco[0]]


O primeiro preço representa o preço cheio, o segundo com desconto e o terceiro é o preço de cada parcela. Este último não representa muita coisa, já que o número de parcelas pode variar entre produtos. Vamos tratar os strings dos dois primeiros, transformado-os em floats, em uma lista de preços:

In [ ]:
import re


In [ ]:
def limpar_preco(str_preco):
    pattern = r"[^0-9.,]"
    return float(re.sub(pattern, "", str_preco).replace(",", "."))


In [ ]:
[limpar_preco(preco.text) for preco in lista_elem_preco[0]]


Agora vamos construir nosso loop!

In [ ]:
lista_preco_cheio = []
lista_preco_desconto = []
pattern = r"[^0-9.,]"

for elem_preco in lista_elem_preco:
    precos = [limpar_preco(preco.text) for preco in elem_preco]
    lista_preco_cheio.append(precos[0])
    lista_preco_desconto.append(precos[1])

print(lista_preco_cheio)


Vamos investigar a lista de preços com desconto:

In [ ]:
print(lista_preco_desconto)

Alguns preços estão muito abaixo do esperado! Se compararmos os resultados com a página veremos que nem todos os produtos tem descontos. Vamos investigar um preço que tenha essa caracteristica:

In [ ]:
[limpar_preco(preco.text) for preco in lista_elem_preco[1]]

Quando um produto não está promocionado, o segundo elemento da lista preço é o valor da parcela para compras parceladas! Vamos alterar nosso loop para contemplar essa condição, guardando `np.nan` no preço descontado de produtos sem desconto:

In [ ]:
lista_preco_cheio = []
lista_preco_desconto = []
pattern = r"[^0-9.,]"

for elem_preco in lista_elem_preco:
    if len(elem_preco) == 3:
        precos = [limpar_preco(preco.text) for preco in elem_preco]
        lista_preco_cheio.append(precos[0])
        lista_preco_desconto.append(precos[1])
    else:
        print("Produto sem promoção!")
        lista_preco_cheio.append(precos[0])
        lista_preco_desconto.append(np.nan)


Podemos utilizar o mesmo método para construir a lista de nomes de produto:

In [ ]:
lista_elem_produto[0].find_element(By.CLASS_NAME, "ui-search-item__title").text


In [ ]:
lista_nome = []
for elem_nome in lista_elem_produto:
    nome = elem_nome.find_element(By.CLASS_NAME, "ui-search-item__title")
    lista_nome.append(nome.text)


Agora vamos juntar nossas duas listas em um DataFrame:

In [ ]:
tb_azeite = pd.DataFrame(
    {
        "nome": lista_nome,
        "preco_cheio": lista_preco_cheio,
        "preco_desconto": lista_preco_desconto,
    }
)
tb_azeite.head()
